<a href="https://colab.research.google.com/github/arjak-roy/Sketch-n-Guess-ML/blob/main/SnG_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading all datasets
**bold text**

In [ ]:
import os
import sys
from google.cloud import storage

def download_public_file(bucket_name, source_blob_name, destination_file_name):
    """Downloads a public blob from the bucket with progress tracking."""

    storage_client = storage.Client.create_anonymous_client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    # Get the size of the blob
    blob.reload()  # Ensures we have the latest metadata
    file_size = blob.size

    if file_size is None:
        print("Could not retrieve file size.")
        return

    # Stream the download
    with open(destination_file_name, "wb") as file:
        with blob.open("rb") as blob_stream:
            downloaded_bytes = 0
            chunk_size = 1024 * 1024  # 1MB chunk size

            while chunk := blob_stream.read(chunk_size):
                file.write(chunk)
                downloaded_bytes += len(chunk)
                progress = (downloaded_bytes / file_size) * 100
                sys.stdout.write(f"\rDownloading... {progress:.2f}% complete")
                sys.stdout.flush()

    print(f"\nDownloaded public blob {source_blob_name} from bucket {bucket_name} to {destination_file_name}.")


In [ ]:
download_public_file("quickdraw_dataset", "full/numpy_bitmap/airplane.npy", "airplane.npy")

Downloading... 100.00% complete
Downloaded public blob full/numpy_bitmap/airplane.npy from bucket quickdraw_dataset to airplane.npy.


In [ ]:
download_public_file("quickdraw_dataset", "full/numpy_bitmap/cat.npy", "cat.npy")

Downloading... 100.00% complete
Downloaded public blob full/numpy_bitmap/cat.npy from bucket quickdraw_dataset to cat.npy.


In [ ]:
download_public_file("quickdraw_dataset", "full/numpy_bitmap/car.npy", "car.npy")

Downloading... 100.00% complete
Downloaded public blob full/numpy_bitmap/car.npy from bucket quickdraw_dataset to car.npy.


In [ ]:
download_public_file("quickdraw_dataset", "full/numpy_bitmap/banana.npy", "banana.npy")

Downloading... 100.00% complete
Downloaded public blob full/numpy_bitmap/banana.npy from bucket quickdraw_dataset to banana.npy.


In [ ]:
import numpy as np
from PIL import Image

airplane = np.load("airplane.npy")


In [ ]:
for i in range(airplane.shape[0]):
  airplane_image = Image.fromarray(airplane[i].reshape(28, 28))
  airplane_image
  binary_image = np.array(airplane_image.convert("L").resize((28, 28)))
  binary_image_array = np.where( binary_image > 10, 255, 0).astype(np.uint8)
  airplane[i] = binary_image_array.flatten()


In [ ]:

import cv2
import numpy as np
from PIL import Image

# Assuming 'airplane' array is already loaded and processed as in the previous code.

for i in range(airplane.shape[0]):
    # Reshape the flattened array back to a 28x28 image
    image_28x28 = airplane[i].reshape(28, 28).astype(np.uint8)

    # Apply Otsu's thresholding
    ret, thresholded_image = cv2.threshold(image_28x28, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Now 'thresholded_image' contains the image after Otsu's thresholding
    # You can further process or save this image

    #Example to display using Pillow (optional):
    #thresholded_image_pil = Image.fromarray(thresholded_image)
    #thresholded_image_pil.show()

    #Update the airplane array with the thresholded image (optional):
    airplane[i] = thresholded_image.flatten()


In [ ]:
Image.fromarray(airplane[100].reshape(28, 28))

In [ ]:
cat = np.load("cat.npy")

In [ ]:
for i in range(cat.shape[0]):
    # Reshape the flattened array back to a 28x28 image
    image_28x28 = cat[i].reshape(28, 28).astype(np.uint8)

    # Apply Otsu's thresholding
    ret, thresholded_image = cv2.threshold(image_28x28, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Now 'thresholded_image' contains the image after Otsu's thresholding
    # You can further process or save this image

    #Example to display using Pillow (optional):
    #thresholded_image_pil = Image.fromarray(thresholded_image)
    #thresholded_image_pil.show()

    #Update the airplane array with the thresholded image (optional):
    cat[i] = thresholded_image.flatten()

In [ ]:
Image.fromarray(cat[105000].reshape(28, 28))

In [ ]:
banana = np.load("banana.npy")

In [ ]:
for i in range(banana.shape[0]):
    # Reshape the flattened array back to a 28x28 image
    image_28x28 = banana[i].reshape(28, 28).astype(np.uint8)

    # Apply Otsu's thresholding
    ret, thresholded_image = cv2.threshold(image_28x28, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Now 'thresholded_image' contains the image after Otsu's thresholding
    # You can further process or save this image

    #Example to display using Pillow (optional):
    #thresholded_image_pil = Image.fromarray(thresholded_image)
    #thresholded_image_pil.show()

    #Update the airplane array with the thresholded image (optional):
    banana[i] = thresholded_image.flatten()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming 'airplane', 'cat', and 'banana' are preprocessed numpy arrays
X = np.concatenate((airplane[:20000], cat[:20000], banana[:20000]), axis=0)
y = np.concatenate((np.zeros(20000), np.ones(20000), np.full(20000, 2)), axis=0)

# Reshape X for LSTM input (samples, timesteps, features)
X = X.reshape(-1, 28, 28)

# One-hot encode the labels for categorical classification
y = to_categorical(y, num_classes=3)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(28, 28)))
model.add(Dense(3, activation='softmax'))  # Softmax for multi-class classification

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.7622 - loss: 0.5692 - val_accuracy: 0.9078 - val_loss: 0.2626
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9259 - loss: 0.2106 - val_accuracy: 0.9377 - val_loss: 0.1767
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.9465 - loss: 0.1525 - val_accuracy: 0.9485 - val_loss: 0.1490
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - accuracy: 0.9572 - loss: 0.1285 - val_accuracy: 0.9574 - val_loss: 0.1262
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9641 - loss: 0.1068 - val_accuracy: 0.9606 - val_loss: 0.1115
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 40s 14ms/step - accuracy: 0.9646 - loss: 0.1005 - val_accuracy: 0.9637 - val_loss: 0.1044
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - accuracy: 0.9684 - loss: 0.0898 - val_accuracy: 0.9707 - val_loss: 0.0845
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 50s 21ms/step - accuracy: 0.9716 -

In [ ]:
Image.fromarray(X_test[0].reshape(28, 28))

In [ ]:
Image.fromarray(X_test[1].reshape(28, 28))

In [ ]:
Image.fromarray(X_test[5].reshape(28, 28))

In [ ]:
result = model.predict(X_test[553].reshape(1, 28, 28))
result.argmax()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


np.int64(0)

In [ ]:
result = model.predict(X_test[1].reshape(1, 28, 28))
result.argmax()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


np.int64(1)

In [ ]:
sample = Image.open("/content/1743862286389.png")
image_array = np.array(sample.convert("L").resize((28, 28,),Image.Resampling.LANCZOS))
ret, thresholded_image = cv2.threshold(image_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
Image.fromarray(thresholded_image)
result = model.predict(thresholded_image.reshape(1, 28, 28))
result.argmax()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


np.int64(2)

In [ ]:
sample = Image.open("/content/plane.png")
image_array = np.array(sample.convert("L").resize((28, 28,),Image.Resampling.LANCZOS))
ret, thresholded_image = cv2.threshold(image_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
Image.fromarray(thresholded_image)
result = model.predict(thresholded_image.reshape(1, 28, 28))
result.argmax()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


np.int64(0)

In [ ]:
def _predict(image):
  result = model.predict(image)
  res = result.argmax()
  if(res == 0):
    return "Plane"
  if(res == 1):
    return "Cat"
  if(res == 2):
    return "Banana"

In [ ]:
_predict(X_test[1].reshape(1, 28, 28))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


'Cat'

In [ ]:
model.save("classifier_sng.h5")